In [1]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [2]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

In [3]:
time.sleep(10)
#Required to activate card-section below
browser.maximize_window()
lt=[t for t in browser.find_elements_by_class_name('card-section') 
   if t.text.find('Latest Activity')>-1][0]

In [5]:
lt.find_element_by_tag_name('a').click()

Web page with latest activity

General info

In [6]:
time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

Load tables

In [7]:
browser.find_element_by_id('show-hidden-efforts').click()

In [8]:
html=browser.page_source

In [9]:
ll=pd.read_html(html)

General info table

In [10]:
mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

In [11]:
mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

In [12]:
mtdt

{'Calories': {'Avg': '611', 'Max': '611'},
 'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:08:13'},
 'Speed': {'Avg': '7.8km/h', 'Max': '57.6km/h'},
 'date': '1:49 PM on Sunday, May 24, 2020',
 'distance': '8.20km',
 'elevation': '447m',
 'energy': '548kJ',
 'name': 'La Tuna',
 'power': '447m',
 'time': '1:03:18'}

Segments

In [13]:
sgmt=ll[1][['Name','Time','Speed','Power']]

In [14]:
sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

In [15]:
sgmt['Info']=''

In [16]:
sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [17]:
sgmt[:1]

,Name,Time,Speed,Power,Info
0,"Loma linda, detrás del antiguo principito 0.4...",2:26,11.4km/h,206W,"{'power': '447m', 'Calories': {'Max': '611', '..."


In [19]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [43]:
df.to_json(db_file)